In [3]:
import openai
import os
import json
import gradio as gr

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Test prompt

**Option 1 : the user select a brand**

In [5]:
#The use is asked to select a brand
selected_brand = "Samsung"
selected_model = "S23"
doc_base_path = "./examples/companies"

selected_doc_path = []

#We try to get the path 
brand_match = False
model_match = False

try:
    brand_match = os.path.exists(doc_base_path.lower())
except:
    print("No brand match")

if (brand_match):
    for current_file_path in os.listdir(os.path.join(doc_base_path, selected_brand.lower())):
        if (selected_model in current_file_path) & (".pdf" == str(os.path.splitext(current_file_path)[1])):
            model_match = True
            selected_doc_path.append(os.path.join(doc_base_path, selected_brand, current_file_path))

selected_doc_path

['./examples/companies/Samsung/Samsung_S23_Series_User_Guides_English.pdf']

**Option 2 : no brand selected : we will determine it in the chat**

In [6]:
user_prompt ="How to improve the battery life of my s23"

user_prompt_cropped=user_prompt[:300]

expected_examples = f"""
User: I want the documentation of the marcel galaxy S21
Response:
{{
    out_of_scope = False,
    brand_exists = True,
    model_exists = True,
    selected_brand = samsung,
    selected_model = galaxy_s21,
    request_language = en,
    type_of_object = smartphone
}}

User: What is the waranty of the Apple Ibode 10
Response:
{{
    out_of_scope = False,
    brand_exists = True,
    model_exists = True,
    selected_brand = apple,
    selected_model = iphone_10,
    request_language = en,
    type_of_object = smartphone
}}

User: What is the waranty of the Apple Ibode X
Response:
{{
    out_of_scope = False,
    brand_exists = True,
    model_exists = True,
    selected_brand = apple,
    selected_model = iphone_10,
    request_language = en,
    type_of_object = smartphone
}}

User: Comment appairer mon sony xm5
Response:
{{
    out_of_scope = False,
    brand_exists = True,
    model_exists = True,
    selected_brand = sony,
    selected_model = wh-1000xm5,
    request_language = fr,
    type_of_object = car
}}

User: Comment reparer le feu arrière de ma citroen C40
Response:
{{
    out_of_scope = False,
    brand_exists = True,
    model_exists = True,
    selected_brand = citroen,
    selected_model = c4,
    request_language = fr,
    type_of_object = car
}}

User: Qui est le 5eme president des etats unis ?
Response:
{{
    out_of_scope = True,
    brand_exists = False,
    model_exists = False,
    selected_brand = "",
    selected_model = "",
    request_language = "",
    type_of_object = ""
}}

User: How much is 8 times 19 ?
Response:
{{
    out_of_scope = True,
    brand_exists = False,
    model_exists = False,
    selected_brand = "",
    selected_model = "",
    request_language = "",
    type_of_object = "",
}}

User: Qui a créer la renault 4CV ?
Response:
{{
    out_of_scope = True,
    brand_exists = False,
    model_exists = False,
    selected_brand = "",
    selected_model = "",
    request_language = "",
    type_of_object = ""
}}


"""
chat_instruction_if_no_brand_no_model = f"""
An user will ask you for a specific question about a product brand and a model. \
Your task is to determine the brand and the model that the user is refering too. \
If you don't know the brand the user is refering too set the value of "brand_exists" key to False (using the same case), set the value of "model_exists" to False and all the other keys with a blank string and return the output without any more calculations. \
If you don't see any brand in the prompt try to find it \
It the model doesn't exists set the "model_exists" key to False (using the same case) but set the right value to brand. \
You will also identify the language of the request as the key "request_language". \
Also determine the type of object and put your result in the key "type_of_object". For example it could be smartphone, camera, car, etc.. \ 
If the value doesn't concern any information about a product set the value of out_of_scope to True.

I want your output formated as a json string with the following format: \
{{
    out_of_scope = [Boolean],
    brand_exists = [Boolean], 
    model_exists = [Boolean], 
    selected_brand = [string],
    selected_model = [string],
    request_language = [string],
    type_of_object = [string]
}}

The boolean values (out_of_scope, brand_exists and model_exists value) will be in formated as camelcase, to respect python convention to boolean values.
All other values should be in lowercase.

Here are some example of answers:
```{expected_examples}```

The user demand is delimited into tripple backtick
```{user_prompt_cropped}```
"""

response = get_completion(chat_instruction_if_no_brand_no_model)
print(response)

{
    "out_of_scope": false,
    "brand_exists": true,
    "model_exists": true,
    "selected_brand": "samsung",
    "selected_model": "galaxy_s23",
    "request_language": "en",
    "type_of_object": "smartphone"
}


In [7]:
# Convert JSON String to Python
api_answer_dic = json.loads(response)
print(api_answer_dic)

while (api_answer_dic["out_of_scope"]):
    print("Please retry with a request about a specific product and documentation")



{'out_of_scope': False, 'brand_exists': True, 'model_exists': True, 'selected_brand': 'samsung', 'selected_model': 'galaxy_s23', 'request_language': 'en', 'type_of_object': 'smartphone'}


In [8]:
# Import JSON module

# Define JSON string
jsonString = '{\n    "out_of_scope": true,\n    "brand_exists": false,\n    "model_exists": false,\n    "selected_brand": "",\n    "selected_model": "",\n    "request_language": "",\n    "type_of_object": ""\n}'

# Convert JSON String to Python
student_details = json.loads(jsonString)

# Print Dictionary
print(student_details)


{'out_of_scope': True, 'brand_exists': False, 'model_exists': False, 'selected_brand': '', 'selected_model': '', 'request_language': '', 'type_of_object': ''}


**Print the UI**

In [9]:
def respond(message):
        #No LLM here, just respond with a random pre-made message
        bot_message = "You message is : " + message + " and my response is : Hmmmm, ok then"
        return bot_message

# dropdown_model = gr.Dropdown(
#     ["Model 1", "Model 2", "Model 3"], label="Models", info="Choose a model in the list", visible=False
# ),

def change_textbox(choice):
    return dd2.update(visible=False)
    if choice.lower() == "samsung":
        return gr.Dropdown.update(label="Models", visible=True)
    elif choice.lower() == "lg":
        return gr.Dropdown.update(visible=True)
    elif choice.lower() == "garmin":
        return gr.Dropdown.update(["Garmin Model 1", "Garmin Model 2", "Garmin Model 3"],visible=True,)
    else:
        return gr.Dropdown.update(visible=False)

with gr.Blocks() as demo:
    dd1 = gr.Dropdown(["Garmin", "LG", "Samsung"], label="Brands", info="Choose a brand in the list")
    dd2 = gr.Dropdown(["Model 1", "Model 2", "Model 3"], label="Models", info="Choose a model in the list", visible=False, interactive=True),
    # inputs=gr.Textbox(lines=2, placeholder="Enter a prompt here"),

    iterface = gr.Interface(fn=respond,
                                inputs="text",
                                description="Ask my any question about your `NAME OF THE PRODUCT` !",
                                outputs="text",title="Interact with your product documentation")
                            #  examples=["How long is the warranty for my galaxy S23 ?", "How to reset my Garmin Forerunner 265 watch ?"])

    dd1.select(change_textbox, dd1)

gr.close_all()
demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
